## 정적 크롤링 (BeautifulSoup)

In [6]:
import requests
from bs4 import BeautifulSoup
import re
import time
import pandas as pd
import numpy as np

# 시작번호 https://www.10000recipe.com/recipe/6947317

start = time.time()

food_list = []
ingredient_list = []
seasoning_list = []
how_to_list = []
how_to_str = []
recipe_num_list = []
ingredient_name = []
ingredient_name_list = []
seasoning_name = []
seasoning_name_list = []
views_list = []
img_url_list = []

# 크롤링 시작할 레시피 번호 선택
recipe_num = 6947317
# 크롤링 확인용
check_num = 1;

while True:
    req = requests.get("https://www.10000recipe.com/recipe/"+ str(recipe_num))
    html = req.text
        # 해당 번호의 레시피가 존재 하지 않을 경우 len(html) < 100이므로  recipe_num -= 1
    if len(html) < 100:
        while True:
            recipe_num -= 1;
            req = requests.get("https://www.10000recipe.com/recipe/"+ str(recipe_num))
            html = req.text
            # 해당 번호의 레시피가 존재시 len(html) > 200이므로 bradk 후 크롤링 시작
            if len(html) > 200:
                break;
        
    bs = BeautifulSoup(html, 'html.parser')
    
    # 크롤링 확인용(크롤링이 오래걸리기 때문에 현재 진행 상황을 보기 위함)
    print(check_num, ':', recipe_num, sep='', end=', ')
    
    # 음식이름
    d_food = bs.select('#contents_area > div.view2_summary.st3 > h3')
    food = [dom.text for dom in d_food]
    food_str = ''.join(food)
    food = []
    food_list.append(food_str)

    # 재료
    d_ingredient = bs.select('#divConfirmedMaterialArea > ul:nth-of-type(1) > a > li')
    ingredient_str = [dom.text for dom in d_ingredient]
    ingredient_new_str = []
        # 재료명과 개수 전처리 (ex: 사과 1개)
    for ingredient in ingredient_str:
        instr = ingredient.replace('\n', '')
        instr = ' '.join(instr.split())
        ingredient_new_str.append(instr)

    ingredient_list.append(ingredient_new_str)
    ingredient_str = []

    
    # 재료명 (유사도검사를 위한)
    inum = 1;
    while True:
        d_ingredient_name = bs.select('#divConfirmedMaterialArea > ul:nth-of-type(1) > a:nth-of-type('+ str(inum) +') > li')
        if not d_ingredient_name :
            break;
        inum += 1;
        ingredient_name_str = [dom.text for dom in d_ingredient_name]
        # 재료명과 개수 분리후 재료명만 선택
        ingredient_name_str = ''.join(ingredient_name_str).split()[0]
        ingredient_name.append(ingredient_name_str)
        ingredient_name_str = []
    ingredient_name = ' '.join(ingredient_name)
    ingredient_name_list.append(ingredient_name)
    ingredient_name = []
    
    # 양념
    d_seasoning = bs.select('#divConfirmedMaterialArea > ul:nth-of-type(2) > a > li')
    seasoning_str = [dom.text for dom in d_seasoning]
    seasoning_new_str = []
        # 양념명과 개수 전처리
    for seasoning in seasoning_str:
        seastr = seasoning.replace('\n', '')
        seastr = ' '.join(seastr.split())
        seasoning_new_str.append(seastr)
    seasoning_list.append(seasoning_new_str)
    seasoning_str = []

    
    # 양념 재료명 (유사도검사를 위한), (실제로는 사용하지 않았음)
    snum = 1;
    while True:
        d_seasoning_name = bs.select('#divConfirmedMaterialArea > ul:nth-of-type(2) > a:nth-of-type('+ str(snum) +') > li')
        if not d_seasoning_name :
            break;
        snum += 1;
        seasoning_name_str = [dom.text for dom in d_seasoning_name]
        seasoning_name_str = ''.join(seasoning_name_str).split()[0]
        seasoning_name.append(seasoning_name_str)
        seasoning_name_str = []
    seasoning_name = ' '.join(seasoning_name)
    seasoning_name_list.append(seasoning_name)
    seasoning_name = []
    

    # 조리방법
        # 조리방법이 ul안에 각각 li로 나눠져 있어 차례대로 긁어오기 위한 cnum
    cnum = 1;
    while True:
        d_howto = bs.select('#stepdescr'+ str(cnum))
        hlist = [dom.text for dom in d_howto]
        # 사이트내에 조리방법 형식이 불일치 하여 값이 없을 경우 break, 후에 전처리로 해당 레시피 행 삭제
        if not hlist:
            break;        
        hlist = hlist[0]
        # 조리방법 양식 통일을 위한 전처리 (ex: 1. 토마토를 물에 씻어주세요)
        howtolist = str(cnum) + '. ' + hlist
        how_to_str.append(howtolist)
        hlist = []
        cnum += 1;
    how_to_list.append(how_to_str)
    how_to_str = []
    
    
    # 조회수
    d_views = bs.select('#contents_area > div.view2_pic > div.view_cate.st2 > div > span')
    views = [dom.text for dom in d_views]
        # 숫자 외에 모든 문자 제거
    views_str = re.sub('[^0-9]', '', str(views))
    views = []
    views_list.append(views_str)
    
    
    # 이미지
    d_img = bs.select('#main_thumbs')
        # 이미지 url만 선택 src="http 부터 " 까지
    img_url = re.findall('src="http.*?"', str(d_img))
        # url만 추출하기 위한 전처리("와 src= 제거)
    img_url[0] = img_url[0].replace('"', '').replace('src=', '')
    img_url_list.append(img_url[0])
    
    
    # 레시피번호
    recipe_num_list.append(recipe_num)
    
    recipe_num -= 1;
    check_num += 1;
    
    # 크롤링 끝낼 번호 설정
    if recipe_num <= 6947307:
        break;
    

# pandas DataFrame만들기
df = pd.DataFrame({"name" : food_list,
                   "ingredient" : ingredient_list,
                   "ingredient_name" : ingredient_name_list,
                   "seasoning" : seasoning_list,
                   "seasoning_name" : seasoning_name_list,
                   "howto" : how_to_list,
                   "views" : views_list,
                   "img" : img_url_list,
                   "recipe_num" : recipe_num_list})  

# DataFrame csv형식으로 저장
df.to_csv("원하는경로와 파일명.csv", index=False)
        
print("time: ", time.time()-start)

1:6947317, 2:6947316, 3:6947315, 4:6947314, 5:6947313, 6:6947312, 7:6947311, 8:6947310, 9:6947309, 10:6947308, time:  9.464988470077515
